In [4]:
from conllu import parse

with open('data/finnish.conllu', 'r', encoding='utf8') as file:
    data = file.read()

sentences = parse(data)

import spacy
from spacy.tokens import Doc, Token
from conllu import parse_incr
from conllu import parse
from spacy.vocab import Vocab
from spacy import displacy
from pprint import pprint

nlp = spacy.blank("fi")  
vocab = nlp.vocab

if not Token.has_extension("xpos"):
    Token.set_extension("xpos", default=None)
if not Token.has_extension("deps"):
    Token.set_extension("deps", default=None)
if not Token.has_extension("misc"):
    Token.set_extension("misc", default=None)


def process_conllu_file(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for conllu_sentence in parse_incr(file):
            doc = conllu_to_spacy_doc(conllu_sentence, vocab)
            docs.append(doc)
    return docs

def conllu_to_spacy_doc(conllu_sentence, vocab):
    words = [token["form"] for token in conllu_sentence]
    heads = [token["head"] if token["head"] is not None else 0 for token in conllu_sentence]  # Head = 0 para raíces
    deps = [token["deprel"].split(":")[0] for token in conllu_sentence]
    lemmas = [token["lemma"] for token in conllu_sentence]
    pos_tags = [token["upostag"] for token in conllu_sentence]
    xpos_tags = [token["xpos"] for token in conllu_sentence]
    morph_feats = [token["feats"] or {} for token in conllu_sentence]
    misc_info = [token["misc"] or {} for token in conllu_sentence]
    deps_info = [token["deps"] or {} for token in conllu_sentence]

    # Crear el objeto Doc
    doc = Doc(vocab, words=words)

    # Asignar los atributos a cada token
    for i, token in enumerate(doc):
        token.lemma_ = lemmas[i]
        
        # Asignar el Universal POS a `pos_`, o usar `tag_` si no es válido
        if pos_tags[i] != "_":
            token.pos_ = pos_tags[i]
        elif xpos_tags[i] is not None:  # Verificar si xpos_tags[i] no es None antes de asignarlo
            token.tag_ = xpos_tags[i]  # Usar `xpos` como `tag_` si `upos` es "_"

        token.dep_ = deps[i]
        
        # Verificar que heads[i] sea un número entero antes de asignarlo como cabeza
        if isinstance(heads[i], int) and heads[i] > 0:
            token.head = doc[heads[i] - 1]  # Usar head como índice (spaCy usa índice 0 para la raíz)
        else:
            token.head = token  # Si head es 0 o no válido, el token se apunta a sí mismo como raíz

        # Asignar xpos, deps y misc como atributos personalizados
        token._.xpos = xpos_tags[i]
        token._.deps = deps_info[i]
        token._.misc = misc_info[i]

        # Convertir `feats` a un formato que spaCy entiende y asignarlo a morph
        if morph_feats[i]:
            feats_str = "|".join(f"{k}={v}" for k, v in morph_feats[i].items())
            token.set_morph(feats_str)

    return doc

def render_doc_by_sent_id(docs, sentences, target_sent_id):
    # Buscar el índice del `sent_id` deseado en los metadatos de `sentences`
    for i, sentence in enumerate(sentences):
        if sentence.metadata.get("sent_id") == target_sent_id:
            # Renderizar el Doc correspondiente usando `displacy`
            displacy.render(docs[i], style="dep", jupyter=True)
            return  # Salimos después de encontrar y renderizar
    print(f"Oración con sent_id '{target_sent_id}' no encontrada.")


file_path = "data/finnish.conllu"
docs = process_conllu_file(file_path)


IN

In [5]:
render_doc_by_sent_id(docs, sentences, '1')
# The cat is in the wardrobe.

print(sentences[0])

for token in sentences[0]:
    print(token.items())

TokenList<Kissa, on, vaatekaapissa, ., metadata={generator: "UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe", udpipe_model: "finnish-tdt-ud-2.12-230717", udpipe_model_licence: "CC BY-NC-SA", newdoc: "None", newpar: "None", sent_id: "1", text: "Kissa on vaatekaapissa."}>
dict_items([('id', 1), ('form', 'Kissa'), ('lemma', 'kissa'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 3), ('deprel', 'nsubj:cop'), ('deps', None), ('misc', {'TokenRange': '0:5'})])
dict_items([('id', 2), ('form', 'on'), ('lemma', 'olla'), ('upos', 'AUX'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 3), ('deprel', 'cop'), ('deps', None), ('misc', {'TokenRange': '6:8'})])
dict_items([('id', 3), ('form', 'vaatekaapissa'), ('lemma', 'vaate#kaappi'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Ine', 'Number': 'Sing'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('mi

In [7]:
render_doc_by_sent_id(docs, sentences, '2')
# She enters the shop.


print(sentences[1])

for token in sentences[1]:
    print(token.items())

TokenList<Hän, menee, kauppaan, ., metadata={sent_id: "2", text: "Hän menee kauppaan."}>
dict_items([('id', 1), ('form', 'Hän'), ('lemma', 'hän'), ('upos', 'PRON'), ('xpos', 'Pron'), ('feats', {'Case': 'Nom', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '24:27'})])
dict_items([('id', 2), ('form', 'menee'), ('lemma', 'mennä'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '28:33'})])
dict_items([('id', 3), ('form', 'kauppaan'), ('lemma', 'kauppa'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Ill', 'Number': 'Sing'}), ('head', 2), ('deprel', 'obl'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '34:42'})])
dict_items([('id', 4), ('form', '.'), ('lemma', '.'), ('upos', 'PUNCT'), ('xpos', 'Punct'), ('feats', None), 

In [8]:
sentences[1][2].get('feats').get('Case')

'Ill'

In [ ]:
render_doc_by_sent_id(docs, sentences, '3')
# He came out of the cave.

# ulos is quite interesing, maybe is literally out?

print(sentences[2])

for token in sentences[2]:
    print(token.items())

TokenList<Hän, tuli, ulos, luolasta, ., metadata={sent_id: "3", text: "Hän tuli ulos luolasta."}>
dict_items([('id', 1), ('form', 'Hän'), ('lemma', 'hän'), ('upos', 'PRON'), ('xpos', 'Pron'), ('feats', {'Case': 'Nom', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '44:47'})])
dict_items([('id', 2), ('form', 'tuli'), ('lemma', 'tulla'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '48:52'})])
dict_items([('id', 3), ('form', 'ulos'), ('lemma', 'ulos'), ('upos', 'ADV'), ('xpos', 'Adv'), ('feats', None), ('head', 2), ('deprel', 'advmod'), ('deps', None), ('misc', {'TokenRange': '53:57'})])
dict_items([('id', 4), ('form', 'luolasta'), ('lemma', 'luola'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Ela', 'Number': 'Sing'}), ('head',

ON

In [ ]:
render_doc_by_sent_id(docs, sentences, '4')
# The book is on the table.


# addesive for [place, ON]

print(sentences[3])

for token in sentences[3]:
    print(token.items())

TokenList<Kirja, on, pöydällä, ., metadata={sent_id: "4", text: "Kirja on pöydällä."}>
dict_items([('id', 1), ('form', 'Kirja'), ('lemma', 'kirja'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 3), ('deprel', 'nsubj:cop'), ('deps', None), ('misc', {'TokenRange': '68:73'})])
dict_items([('id', 2), ('form', 'on'), ('lemma', 'olla'), ('upos', 'AUX'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 3), ('deprel', 'cop'), ('deps', None), ('misc', {'TokenRange': '74:76'})])
dict_items([('id', 3), ('form', 'pöydällä'), ('lemma', 'pöytä'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Ade', 'Number': 'Sing'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '77:85'})])
dict_items([('id', 4), ('form', '.'), ('lemma', '.'), ('upos', 'PUNCT'), ('xpos', 'Punct'), ('feats', None), ('head', 3), ('deprel', 'punct'), ('d

In [ ]:
render_doc_by_sent_id(docs, sentences, '5')
# The boy jumped onto the table.

# allative for [place, ON]

print(sentences[4])

for token in sentences[4]:
    print(token.items())

TokenList<Poika, hyppäsi, pöydälle, ., metadata={sent_id: "5", text: "Poika hyppäsi pöydälle."}>
dict_items([('id', 1), ('form', 'Poika'), ('lemma', 'poika'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '87:92'})])
dict_items([('id', 2), ('form', 'hyppäsi'), ('lemma', 'hypätä'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '93:100'})])
dict_items([('id', 3), ('form', 'pöydälle'), ('lemma', 'pöytä'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'All', 'Number': 'Sing'}), ('head', 2), ('deprel', 'obl'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '101:109'})])
dict_items([('id', 4), ('form', '.'), ('lemma', '.'), ('upos', 'PUNCT'), ('xpos', 'Punct'), ('feats', None), ('head', 2), ('depre

In [ ]:
render_doc_by_sent_id(docs, sentences, '6')
# The girl climbed down from the table.


print(sentences[5])

for token in sentences[5]:
    print(token.items())

TokenList<Tyttö, kiipesi, alas, pöydältä, ., metadata={sent_id: "6", text: "Tyttö kiipesi alas pöydältä."}>
dict_items([('id', 1), ('form', 'Tyttö'), ('lemma', 'tyttö'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '111:116'})])
dict_items([('id', 2), ('form', 'kiipesi'), ('lemma', 'kiivetä'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '117:124'})])
dict_items([('id', 3), ('form', 'alas'), ('lemma', 'alas'), ('upos', 'ADV'), ('xpos', 'Adv'), ('feats', None), ('head', 2), ('deprel', 'advmod'), ('deps', None), ('misc', {'TokenRange': '125:129'})])
dict_items([('id', 4), ('form', 'pöydältä'), ('lemma', 'pöytä'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Abl', 'Number': 'Sing'}), ('head', 2), ('depre

AT

In [16]:
render_doc_by_sent_id(docs, sentences, '7')
# The bird rests next to the tree.


print(sentences[6])

for token in sentences[6]:
    print(token.items())


TokenList<Lintu, lepää, puun, vieressä, ., metadata={sent_id: "7", text: "Lintu lepää puun vieressä."}>
dict_items([('id', 1), ('form', 'Lintu'), ('lemma', 'lintu'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '140:145'})])
dict_items([('id', 2), ('form', 'lepää'), ('lemma', 'levätä'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '146:151'})])
dict_items([('id', 3), ('form', 'puun'), ('lemma', 'puu'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Gen', 'Number': 'Sing'}), ('head', 2), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '152:156'})])
dict_items([('id', 4), ('form', 'vieressä'), ('lemma', 'vieressä'), ('upos', 'ADP'), ('xpos', 'Adp'), ('feats', {'AdpType': 'Post'}), ('head', 3), 

In [17]:
render_doc_by_sent_id(docs, sentences, '8')
# The boy runs toward the tree.



print(sentences[7])

for token in sentences[7]:
    print(token.items())

TokenList<Poika, juoksee, puuta, kohti, ., metadata={sent_id: "8", text: "Poika juoksee puuta kohti."}>
dict_items([('id', 1), ('form', 'Poika'), ('lemma', 'poika'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '167:172'})])
dict_items([('id', 2), ('form', 'juoksee'), ('lemma', 'juosta'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '173:180'})])
dict_items([('id', 3), ('form', 'puuta'), ('lemma', 'puu'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Par', 'Number': 'Sing'}), ('head', 2), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '181:186'})])
dict_items([('id', 4), ('form', 'kohti'), ('lemma', 'kohti'), ('upos', 'ADP'), ('xpos', 'Adp'), ('feats', {'AdpType': 'Post'}), ('head', 3), ('d

In [18]:
render_doc_by_sent_id(docs, sentences, '9')
# The squirrel moved away from the tree.



print(sentences[8])

for token in sentences[8]:
    print(token.items())

TokenList<Orava, liikkui, pois, puun, luota, ., metadata={sent_id: "9", text: "Orava liikkui pois puun luota."}>
dict_items([('id', 1), ('form', 'Orava'), ('lemma', 'orava'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '194:199'})])
dict_items([('id', 2), ('form', 'liikkui'), ('lemma', 'liikkua'), ('upos', 'VERB'), ('xpos', 'V'), ('feats', {'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '200:207'})])
dict_items([('id', 3), ('form', 'pois'), ('lemma', 'pois'), ('upos', 'ADV'), ('xpos', 'Adv'), ('feats', None), ('head', 2), ('deprel', 'advmod'), ('deps', None), ('misc', {'TokenRange': '208:212'})])
dict_items([('id', 4), ('form', 'puun'), ('lemma', 'puu'), ('upos', 'NOUN'), ('xpos', 'N'), ('feats', {'Case': 'Gen', 'Number': 'Sing'}), ('head', 2), ('deprel